In [1]:
import pandas as pd
import numpy as np
import db_utils, sqlalchemy
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import seaborn as sns
from scipy.stats import fisher_exact
from scipy import stats
import math
import os
import gzip
import re
import glob
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import math
def count_rows(group):
    return group.shape[0]

In [2]:
eng = sqlalchemy.create_engine('mysql://', creator= db_utils.get_connection)
query= pd.read_sql_query('''select * from chopyan_db.3did_agg_DDI_PDB as df''', con= eng)
DDI_CS = pd.DataFrame(query)
DDI_CS

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,DomainStart2,DomainEnd2,Score,Zscore,Chain1Iface,Chain2Iface,isInterchain
0,PF00001_PF00001,1gzm,A,PF00001,7tm_1,54,306,B,PF00001,7tm_1,54,306,26.18,7.82923,"E:196,E:196,E:196,E:196,E:197,N:199,E:201,E:20...","F:228,K:231,E:232,A:233,E:232,F:228,V:227,F:22...",1
1,PF00001_PF00001,1hzx,B,PF00001,7tm_1,54,246,B,PF00001,7tm_1,224,306,42.49,7.03413,"I:54,I:54,I:54,N:55,N:55,N:55,T:58,L:72,L:72,L...","V:300,P:303,V:304,A:299,V:300,P:303,Y:306,V:25...",0
2,PF00001_PF00001,1l9h,B,PF00001,7tm_1,54,246,B,PF00001,7tm_1,224,306,40.17,9.39410,"I:54,I:54,N:55,N:55,N:55,T:58,L:72,L:72,L:72,L...","V:300,P:303,A:299,V:300,P:303,Y:306,V:250,M:25...",0
3,PF00001_PF00001,2j4y,A,PF00001,7tm_1,54,306,B,PF00001,7tm_1,54,306,26.85,6.86878,"N:199,E:201,E:201,E:201,S:202,S:202,I:205,V:20...","Q:236,F:228,K:231,E:232,F:228,E:232,F:228,F:22...",1
4,PF00001_PF00001,2vt4,C,PF00001,7tm_1,58,343,D,PF00001,7tm_1,58,343,20.02,6.36751,"L:152,V:160,T:164,T:164,A:167,I:168,I:168,A:17...","F:315,A:206,A:206,A:210,W:181,A:210,I:214,W:18...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235443,PF18841_PF17433,1wmr,A,PF18841,B_solenoid_dext,321,354,A,PF17433,Glyco_hydro_49N,19,184,9.52,4.52235,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235444,PF18841_PF17433,1x0c,B,PF18841,B_solenoid_dext,321,354,B,PF17433,Glyco_hydro_49N,19,184,9.52,4.45403,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235445,PF18841_PF17433,2z8g,A,PF18841,B_solenoid_dext,321,354,A,PF17433,Glyco_hydro_49N,19,184,9.24,4.49346,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235446,PF18841_PF17433,3wwg,C,PF18841,B_solenoid_dext,321,354,C,PF17433,Glyco_hydro_49N,19,184,9.24,4.61412,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0


In [3]:
DDI_CS['PDB_ID']= DDI_CS['PDB_ID'].str.upper()

In [4]:
contacts1 = []
for i in DDI_CS['Chain1Iface']:
    contacts1.append(len(i.split(',')))
DDI_CS['contacts1'] = contacts1
contacts2 = []
for i in DDI_CS['Chain2Iface']:
    contacts2.append(len(i.split(',')))
DDI_CS['contacts2'] = contacts2
DDI_CS[(DDI_CS['contacts1'] != DDI_CS['contacts2'])]

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,DomainStart2,DomainEnd2,Score,Zscore,Chain1Iface,Chain2Iface,isInterchain,contacts1,contacts2


In [5]:
DDI_CS = DDI_CS.drop(columns= ['contacts2'])

In [6]:
df_protein_info = pd.read_csv('/Users/johgeist/Downloads/DDI_type_structure_level_interchain_homo_protein_whole_3did.csv', index_col=0)
df_protein_info= df_protein_info.drop(columns=['IsInterchain', 'Chain1', 'Chain2'])
DDI_CS= pd.merge(DDI_CS, df_protein_info, on=['DDI_type', 'PDB_ID'])

In [7]:
DDI_CS.columns

Index(['DDI_type', 'PDB_ID', 'Chain_ID1', 'DomainID1', 'DomainName1',
       'DomainStart1', 'DomainEnd1', 'Chain_ID2', 'DomainID2', 'DomainName2',
       'DomainStart2', 'DomainEnd2', 'Score', 'Zscore', 'Chain1Iface',
       'Chain2Iface', 'isInterchain', 'contacts1', 'HomoDDI', 'UniProt1',
       'UniProt2', 'HomoProtein'],
      dtype='object')

In [8]:
DDI_inter= DDI_CS.drop(index= DDI_CS[DDI_CS['isInterchain'] == 0].index)
DDI_inter_hetero = DDI_inter.drop(index= DDI_inter[DDI_inter['HomoProtein'] == 1].index)

In [9]:
DDI_inter_hetero.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_metrics_inter_hetero_subset_structure_level.csv', index=False)

In [10]:
#DDI_inter_hetero_grouped = DDI_inter_hetero.groupby('DDI_type').agg({'Score': max, 'Zscore': max, 'contacts1': max}) #1
df_index_max_Score = DDI_inter_hetero.groupby('DDI_type').agg({'Score': 'idxmax'})

DDI_inter_hetero_grouped_max_Score = DDI_inter_hetero[DDI_inter_hetero.index.isin(df_index_max_Score.Score.tolist())] #2

In [11]:
DDI_inter_hetero_grouped_max_Score

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,...,Score,Zscore,Chain1Iface,Chain2Iface,isInterchain,contacts1,HomoDDI,UniProt1,UniProt2,HomoProtein
72,PF00001_PF00048,5UIW,A,PF00001,7tm_1,47,297,B,PF00048,IL8,...,13.26,5.101720,"Y:89,Q:170,Q:170,K:171,K:171,E:172,E:172,E:172...","Q:8,F:28,Y:29,E:26,F:28,Q:8,S:9,F:28,S:31,Q:48...",1,51,0,"P51681,P00268",P13501,0
82,PF00001_PF00085,6H7O,B,PF00001,7tm_1,58,343,F,PF00085,Thioredoxin,...,1.57,1.963520,"V:103,V:103,R:104,R:104,R:104,R:104,R:104","G:51,K:52,D:47,E:48,Y:49,Q:50,G:51",1,7,0,P07700,P0AA25,0
84,PF00001_PF00087,6WJC,A,PF00001,7tm_1,42,418,C,PF00087,Toxin_TOLIP,...,15.76,5.828060,"Y:82,W:164,W:164,L:167,L:167,V:168,V:168,V:168...","R:34,I:9,W:10,W:10,F:11,I:9,W:10,F:11,P:12,R:4...",1,44,0,"P11229,D9IEF7",Q8QGR0,0
85,PF00001_PF00322,5GLH,A,PF00001,7tm_1,107,387,B,PF00322,Endothelin,...,31.19,7.365980,"I:157,N:158,N:158,N:158,K:161,K:161,K:161,K:16...","I:20,I:19,I:20,W:21,C:15,H:16,L:17,D:18,I:19,H...",1,78,0,"P00720,P24530",P05305,0
92,PF00001_PF00339,6TKO,A,PF00001,7tm_1,58,343,B,PF00339,Arrestin_N,...,18.24,5.657100,"R:71,Q:73,T:74,L:75,L:75,T:76,N:77,I:80,R:139,...","D:67,T:136,T:136,E:66,T:136,E:66,E:66,V:70,E:6...",1,38,0,P07700,P49407,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235247,PF18782_PF00559,6NIL,A,PF18782,NAD2,193,372,C,PF00559,Vif,...,7.74,3.639300,"R:252,L:255,S:256,S:256,C:259,D:260,D:260,D:26...","P:49,W:70,H:43,W:70,Q:83,R:15,N:19,K:22,Q:83,R...",1,23,0,Q8IUX4,P12504,0
235281,PF18784_PF00071,3NBZ,D,PF18784,CRM1_repeat_2,405,472,F,PF00071,Ras,...,8.76,4.306780,"E:429,L:431,L:431,L:431,V:433,E:434,D:436,D:43...","N:156,S:153,N:154,Y:155,S:153,R:29,K:37,K:152,...",1,25,0,Q6P5F9,P62826,0
235371,PF18801_PF00072,3Q15,A,PF18801,RapH_N,4,65,C,PF00072,Response_reg,...,10.48,4.961570,"E:45,D:46,Q:47,Q:47,Q:47,Q:47,D:48,L:50,L:50,L...","Y:84,Y:84,Q:12,K:56,A:83,Y:84,Q:12,A:83,Y:84,K...",1,22,0,Q59HN8,P06628,0
235372,PF18806_PF00071,2X19,B,PF18806,Importin_rep_3,826,900,A,PF00071,Ras,...,0.05,0.237612,"P:829,E:830,E:830,E:830,K:835,S:880,S:880","Y:41,K:40,Y:41,I:42,T:95,S:96,I:98",1,7,0,O94829,P32835,0


In [12]:
DDI_inter_hetero_grouped_max_Score[(DDI_inter_hetero_grouped_max_Score['UniProt1']== 'No UniProt') & (DDI_inter_hetero_grouped_max_Score['UniProt2']== 'No UniProt')]

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,...,Score,Zscore,Chain1Iface,Chain2Iface,isInterchain,contacts1,HomoDDI,UniProt1,UniProt2,HomoProtein


In [13]:
DDI_inter_hetero_grouped_max_Score = DDI_inter_hetero_grouped_max_Score.filter(['DDI_type', 'Score', 'Zscore', 'contacts1', 'PDB_ID'])

In [14]:
DDI_inter_hetero_grouped_max_Score

,DDI_type,Score,Zscore,contacts1,PDB_ID
72,PF00001_PF00048,13.26,5.101720,51,5UIW
82,PF00001_PF00085,1.57,1.963520,7,6H7O
84,PF00001_PF00087,15.76,5.828060,44,6WJC
85,PF00001_PF00322,31.19,7.365980,78,5GLH
92,PF00001_PF00339,18.24,5.657100,38,6TKO
...,...,...,...,...,...
235247,PF18782_PF00559,7.74,3.639300,23,6NIL
235281,PF18784_PF00071,8.76,4.306780,25,3NBZ
235371,PF18801_PF00072,10.48,4.961570,22,3Q15
235372,PF18806_PF00071,0.05,0.237612,7,2X19


In [15]:
DDI_inter_hetero_grouped_max_Score.rename({'PDB_ID': 'PDB_ID_max'}, inplace=True, axis=1)

In [16]:
DDI_merged = pd.merge(DDI_CS.groupby('DDI_type').agg({'DomainID1': 'first','DomainID2': 'first',
                                                      'DomainName1': 'first','DomainName2': 'first',
                                                      'PDB_ID': count_rows,
                                                      'isInterchain': sum}).reset_index(),
                                                          DDI_inter_hetero_grouped_max_Score,
                                                        on='DDI_type', how= 'inner')

In [17]:
DDI_merged.rename({'PDB_ID': 'Num_structures', 'isInterchain': 'Num_interchain_structures',
                   'HomoProtein': 'Num_homo_protein_structures', 'HomoDDI': 'Homo_DDI'}, inplace=True, axis=1)
DDI_merged['Interchain_fraction'] = DDI_merged['Num_interchain_structures']/DDI_merged['Num_structures']
DDI_merged['Num_intrachain_structures']= DDI_merged['Num_structures']-DDI_merged['Num_interchain_structures']

In [18]:
DDI_merged

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures
0,PF00001_PF00048,PF00001,PF00048,7tm_1,IL8,8,8,13.26,5.101720,51,5UIW,1.000000,0
1,PF00001_PF00085,PF00001,PF00085,7tm_1,Thioredoxin,7,6,1.57,1.963520,7,6H7O,0.857143,1
2,PF00001_PF00087,PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.828060,44,6WJC,1.000000,0
3,PF00001_PF00322,PF00001,PF00322,7tm_1,Endothelin,4,4,31.19,7.365980,78,5GLH,1.000000,0
4,PF00001_PF00339,PF00001,PF00339,7tm_1,Arrestin_N,6,3,18.24,5.657100,38,6TKO,0.500000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,PF18782_PF00559,PF18782,PF00559,NAD2,Vif,1,1,7.74,3.639300,23,6NIL,1.000000,0
5720,PF18784_PF00071,PF18784,PF00071,CRM1_repeat_2,Ras,34,34,8.76,4.306780,25,3NBZ,1.000000,0
5721,PF18801_PF00072,PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.961570,22,3Q15,1.000000,0
5722,PF18806_PF00071,PF18806,PF00071,Importin_rep_3,Ras,1,1,0.05,0.237612,7,2X19,1.000000,0


In [19]:
DDI_merged.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_metrics_inter_hetero_subset.csv', index=False)

### Add ProtCID features
- The DDI subset of the ProtCID database includes cluster information of all 

In [20]:
eng = sqlalchemy.create_engine('mysql://', creator= db_utils.get_connection)
query= pd.read_sql_query('''select * from chopyan_db.DDI_type_features as df''', con= eng)
DDI_type_feat= pd.DataFrame(query)
DDI_type_feat

,DDI_type,DDI_type_sorted,PfamID1,DomainID1,DomainName1,PfamID2,DomainID2,DomainName2,HuRIpvalue,HuRIzscore,BioPlexpvalue,BioPlexzscore,StructureScore,DDI_type_score,ManuallyCurated,homoDDI,hasInterchain
0,PF10417_PF10417,PF10417_PF10417,PF10417,1-cysPrx_C,C-terminal domain of 1-Cys peroxiredoxin,PF10417,1-cysPrx_C,C-terminal domain of 1-Cys peroxiredoxin,0.000,17.704923,0.000,63.245561,None,NaN,0,1,1
1,PF10417_PF00578,PF00578_PF10417,PF10417,1-cysPrx_C,C-terminal domain of 1-Cys peroxiredoxin,PF00578,AhpC-TSA,AhpC/TSA family,0.000,17.704923,0.000,63.245561,None,0.646112,0,0,1
2,PF10417_PF02195,PF02195_PF10417,PF10417,1-cysPrx_C,C-terminal domain of 1-Cys peroxiredoxin,PF02195,ParBc,ParB-like nuclease domain,NaN,NaN,1.000,-0.031639,None,0.896050,0,0,1
3,PF10417_PF00085,PF00085_PF10417,PF10417,1-cysPrx_C,C-terminal domain of 1-Cys peroxiredoxin,PF00085,Thioredoxin,Thioredoxin,0.039,3.293199,0.035,5.250850,None,0.254769,0,0,1
4,PF12574_PF12574,PF12574_PF12574,PF12574,120_Rick_ant,120 KDa Rickettsia surface antigen,PF12574,120_Rick_ant,120 KDa Rickettsia surface antigen,NaN,NaN,NaN,NaN,None,NaN,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14967,PF05707_PF05707,PF05707_PF05707,PF05707,Zot,Zonular occludens toxin (Zot),PF05707,Zot,Zonular occludens toxin (Zot),NaN,NaN,NaN,NaN,None,NaN,0,1,1
14968,PF16916_PF16916,PF16916_PF16916,PF16916,ZT_dimer,Dimerisation domain of Zinc Transporter,PF16916,ZT_dimer,Dimerisation domain of Zinc Transporter,NaN,NaN,NaN,NaN,None,NaN,0,1,1
14969,PF00791_PF00791,PF00791_PF00791,PF00791,ZU5,ZU5 domain,PF00791,ZU5,ZU5 domain,1.000,-0.100504,1.000,-0.110208,None,NaN,0,1,1
14970,PF09817_PF09817,PF09817_PF09817,PF09817,Zwilch,"RZZ complex, subunit zwilch",PF09817,Zwilch,"RZZ complex, subunit zwilch",NaN,NaN,1.000,0.000000,None,NaN,0,1,1


In [21]:
feat= DDI_type_feat.filter(['DDI_type', 'HuRIzscore', 'BioPlexzscore'])
feat

,DDI_type,HuRIzscore,BioPlexzscore
0,PF10417_PF10417,17.704923,63.245561
1,PF10417_PF00578,17.704923,63.245561
2,PF10417_PF02195,NaN,-0.031639
3,PF10417_PF00085,3.293199,5.250850
4,PF12574_PF12574,NaN,NaN
...,...,...,...
14967,PF05707_PF05707,NaN,NaN
14968,PF16916_PF16916,NaN,NaN
14969,PF00791_PF00791,-0.100504,-0.110208
14970,PF09817_PF09817,NaN,0.000000


In [22]:
df_3did_HuRI_BioPlex = pd.merge(DDI_merged, feat, how= 'inner')

In [23]:
df_3did_HuRI_BioPlex

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures,HuRIzscore,BioPlexzscore
0,PF00001_PF00048,PF00001,PF00048,7tm_1,IL8,8,8,13.26,5.101720,51,5UIW,1.000000,0,8.015862,-2.337582
1,PF00001_PF00085,PF00001,PF00085,7tm_1,Thioredoxin,7,6,1.57,1.963520,7,6H7O,0.857143,1,-0.455672,-1.268307
2,PF00001_PF00087,PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.828060,44,6WJC,1.000000,0,-0.077693,NaN
3,PF00001_PF00322,PF00001,PF00322,7tm_1,Endothelin,4,4,31.19,7.365980,78,5GLH,1.000000,0,10.129397,-0.495000
4,PF00001_PF00339,PF00001,PF00339,7tm_1,Arrestin_N,6,3,18.24,5.657100,38,6TKO,0.500000,3,12.478734,1.637844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,PF18782_PF00559,PF18782,PF00559,NAD2,Vif,1,1,7.74,3.639300,23,6NIL,1.000000,0,NaN,NaN
5720,PF18784_PF00071,PF18784,PF00071,CRM1_repeat_2,Ras,34,34,8.76,4.306780,25,3NBZ,1.000000,0,-0.309397,-0.216169
5721,PF18801_PF00072,PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.961570,22,3Q15,1.000000,0,NaN,NaN
5722,PF18806_PF00071,PF18806,PF00071,Importin_rep_3,Ras,1,1,0.05,0.237612,7,2X19,1.000000,0,2.846743,-0.351625


In [24]:
df_3did_HuRI_BioPlex.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_Interactome_metrics_inter_hetero_subset.csv', index=False)

In [25]:
df_counts = pd.read_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/ProtCID_parsed_files/Master_counts_df_ProtCID_20231017.csv')

In [26]:
df_counts

,DomainNames,FileName,DDI_type,ClusterID,SurfaceArea,MinSeqIdentity,MediumSurfaceArea,PDB_ID_max,UnpCode,PdbID,CFID
0,Med21 Med4,(Med21)(Med4)_19935.txt,PF11221_PF10018,1,481.3500,27.0,506.75,5n9j,2,2,2
1,PI3Ka SH2,(PI3Ka)(SH2)_13501.txt,PF00613_PF00017,1,382.7833,97.0,347.40,5swr,1,30,5
2,Pox_Rap94 RNA_pol_Rpb2_7,(Pox_Rap94)(RNA_pol_Rpb2_7)_35453.txt,PF03294_PF04560,1,480.2750,99.0,527.35,6rfl,2,2,2
3,Adeno_hexon_C Adeno_PVIII,(Adeno_hexon_C)(Adeno_PVIII)_1493.txt,PF01310_PF03678,1,396.3250,33.0,408.70,6cgv,6,7,7
4,Adeno_hexon_C Adeno_PVIII,(Adeno_hexon_C)(Adeno_PVIII)_1493.txt,PF01310_PF03678,2,298.4818,25.0,374.75,6cgv,5,6,6
...,...,...,...,...,...,...,...,...,...,...,...
4343,Fer4_7 NADH_Oxid_Nqo15,(Fer4_7)(NADH_Oxid_Nqo15)_8080.txt,PF12838_PF11497,1,323.8731,99.0,326.20,4hea,1,14,8
4344,Chordopox_RPO7 Pox_RNA_pol_35,(Chordopox_RPO7)(Pox_RNA_pol_35)_35486.txt,PF05864_PF03396,1,1492.7250,98.0,1503.20,6rie,2,4,4
4345,ig MHC_I,(ig)(MHC_I)_18510.txt,PF00129_PF00047,1,311.3024,79.0,297.50,5t70,9,13,12
4346,ig MHC_I,(ig)(MHC_I)_18510.txt,PF00129_PF00047,2,243.0559,30.0,231.25,5t70,7,11,10


In [27]:
df_counts_grouped_max_or = df_counts.groupby('DDI_type').agg({'CFID': max, 'MinSeqIdentity': min,
                                                              'UnpCode': max, 'ClusterID': max})
df_counts_grouped_max_or = df_counts_grouped_max_or.rename(columns ={'CFID': 'Num crystal forms',
                                                                      'MinSeqIdentity': 'Min seq identity',
                                                                     'UnpCode': 'Num of proteins', 
                                                                      'ClusterID': 'Num clusters'})
df_counts_grouped_max_or.reset_index(inplace=True)

In [28]:
df_3did_HuRI_BioPlex['DDI_type']=df_3did_HuRI_BioPlex['DDI_type'].astype('str')
df_counts_grouped_max_or['DDI_type']=df_counts_grouped_max_or['DDI_type'].astype('str')

In [29]:
df_3did_HuRI_BioPlex_ProtCID = df_3did_HuRI_BioPlex.merge(df_counts_grouped_max_or,on= 'DDI_type', how= 'left')

In [30]:
df_3did_HuRI_BioPlex_ProtCID[df_3did_HuRI_BioPlex_ProtCID['Num crystal forms'].isna()]

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters
2,PF00001_PF00087,PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.82806,44,6WJC,1.0,0,-0.077693,NaN,NaN,NaN,NaN,NaN
5,PF00001_PF00400,PF00001,PF00400,7tm_1,WD40,2,2,2.95,2.73640,5,6OSA,1.0,0,0.456522,-4.290103,NaN,NaN,NaN,NaN
16,PF00002_PF04901,PF00002,PF04901,7tm_2,RAMP,4,4,29.41,7.42986,59,6UUN,1.0,0,7.604178,-0.600940,NaN,NaN,NaN,NaN
20,PF00004_PF00101,PF00004,PF00101,AAA,RuBisCO_small,1,1,5.34,3.32008,16,6Z1F,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN
21,PF00004_PF00227,PF00004,PF00227,AAA,Proteasome,5,5,2.37,2.35372,10,1E94,1.0,0,2.041022,10.089070,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5716,PF18773_PF02792,PF18773,PF02792,Importin_rep,Mago_nashi,1,1,5.61,4.50071,10,2X1G,1.0,0,10.493384,0.000000,NaN,NaN,NaN,NaN
5718,PF18779_PF00074,PF18779,PF00074,LRR_RI_capping,RnaseA,1,1,6.45,4.17083,11,2BEX,1.0,0,36.514878,30.811234,NaN,NaN,NaN,NaN
5719,PF18782_PF00559,PF18782,PF00559,NAD2,Vif,1,1,7.74,3.63930,23,6NIL,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN
5721,PF18801_PF00072,PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.96157,22,3Q15,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_3did_HuRI_BioPlex_ProtCID['Num clusters'].fillna(0, inplace=True)
df_3did_HuRI_BioPlex_ProtCID['Min seq identity'].fillna(100, inplace=True)
df_3did_HuRI_BioPlex_ProtCID['Num crystal forms'].fillna(1, inplace=True)
df_3did_HuRI_BioPlex_ProtCID['Num of proteins'].fillna(1, inplace=True) # ask Katja again!!

In [32]:
df_3did_HuRI_BioPlex_ProtCID

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters
0,PF00001_PF00048,PF00001,PF00048,7tm_1,IL8,8,8,13.26,5.101720,51,5UIW,1.000000,0,8.015862,-2.337582,5.0,18.0,4.0,1.0
1,PF00001_PF00085,PF00001,PF00085,7tm_1,Thioredoxin,7,6,1.57,1.963520,7,6H7O,0.857143,1,-0.455672,-1.268307,2.0,95.0,2.0,4.0
2,PF00001_PF00087,PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.828060,44,6WJC,1.000000,0,-0.077693,NaN,1.0,100.0,1.0,0.0
3,PF00001_PF00322,PF00001,PF00322,7tm_1,Endothelin,4,4,31.19,7.365980,78,5GLH,1.000000,0,10.129397,-0.495000,3.0,57.0,3.0,1.0
4,PF00001_PF00339,PF00001,PF00339,7tm_1,Arrestin_N,6,3,18.24,5.657100,38,6TKO,0.500000,3,12.478734,1.637844,5.0,18.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,PF18782_PF00559,PF18782,PF00559,NAD2,Vif,1,1,7.74,3.639300,23,6NIL,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5720,PF18784_PF00071,PF18784,PF00071,CRM1_repeat_2,Ras,34,34,8.76,4.306780,25,3NBZ,1.000000,0,-0.309397,-0.216169,10.0,50.0,9.0,1.0
5721,PF18801_PF00072,PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.961570,22,3Q15,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5722,PF18806_PF00071,PF18806,PF00071,Importin_rep_3,Ras,1,1,0.05,0.237612,7,2X19,1.000000,0,2.846743,-0.351625,2.0,22.0,2.0,1.0


In [33]:
df_3did_HuRI_BioPlex_ProtCID.columns

Index(['DDI_type', 'DomainID1', 'DomainID2', 'DomainName1', 'DomainName2',
       'Num_structures', 'Num_interchain_structures', 'Score', 'Zscore',
       'contacts1', 'PDB_ID_max', 'Interchain_fraction',
       'Num_intrachain_structures', 'HuRIzscore', 'BioPlexzscore',
       'Num crystal forms', 'Min seq identity', 'Num of proteins',
       'Num clusters'],
      dtype='object')

In [34]:
df_3did_HuRI_BioPlex_ProtCID.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_Interactome_ProtCID_metrics_inter_hetero_subset.csv', index=False)

In [35]:
#create 3did subset for representativity analysis

In [36]:
# generate new 3did subset without synthetic construct domains Ank PF00023 (14) PF12796(48) PF13606(1) PF13637(15) PF13657(1) , V-set PF07686 (79)
syn_cons_domains = ['PF00023', 'PF07686', 'PF12796', 'PF13606', 'PF13637', 'PF13657']
final_subset = df_3did_HuRI_BioPlex_ProtCID.copy()
final_subset['DDI_type'] = final_subset['DDI_type'].str.split('_')
rows_to_drop = final_subset.apply(lambda row: any(set(syn_cons_domains).intersection(row['DDI_type'])), axis=1)
final_subset = final_subset[~rows_to_drop]

In [37]:
final_subset

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters
0,"[PF00001, PF00048]",PF00001,PF00048,7tm_1,IL8,8,8,13.26,5.101720,51,5UIW,1.000000,0,8.015862,-2.337582,5.0,18.0,4.0,1.0
1,"[PF00001, PF00085]",PF00001,PF00085,7tm_1,Thioredoxin,7,6,1.57,1.963520,7,6H7O,0.857143,1,-0.455672,-1.268307,2.0,95.0,2.0,4.0
2,"[PF00001, PF00087]",PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.828060,44,6WJC,1.000000,0,-0.077693,NaN,1.0,100.0,1.0,0.0
3,"[PF00001, PF00322]",PF00001,PF00322,7tm_1,Endothelin,4,4,31.19,7.365980,78,5GLH,1.000000,0,10.129397,-0.495000,3.0,57.0,3.0,1.0
4,"[PF00001, PF00339]",PF00001,PF00339,7tm_1,Arrestin_N,6,3,18.24,5.657100,38,6TKO,0.500000,3,12.478734,1.637844,5.0,18.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,"[PF18782, PF00559]",PF18782,PF00559,NAD2,Vif,1,1,7.74,3.639300,23,6NIL,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5720,"[PF18784, PF00071]",PF18784,PF00071,CRM1_repeat_2,Ras,34,34,8.76,4.306780,25,3NBZ,1.000000,0,-0.309397,-0.216169,10.0,50.0,9.0,1.0
5721,"[PF18801, PF00072]",PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.961570,22,3Q15,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5722,"[PF18806, PF00071]",PF18806,PF00071,Importin_rep_3,Ras,1,1,0.05,0.237612,7,2X19,1.000000,0,2.846743,-0.351625,2.0,22.0,2.0,1.0


In [38]:
final_subset['DDI_type'] = final_subset['DDI_type'].str.join('_')

In [39]:
final_subset

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters
0,PF00001_PF00048,PF00001,PF00048,7tm_1,IL8,8,8,13.26,5.101720,51,5UIW,1.000000,0,8.015862,-2.337582,5.0,18.0,4.0,1.0
1,PF00001_PF00085,PF00001,PF00085,7tm_1,Thioredoxin,7,6,1.57,1.963520,7,6H7O,0.857143,1,-0.455672,-1.268307,2.0,95.0,2.0,4.0
2,PF00001_PF00087,PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.828060,44,6WJC,1.000000,0,-0.077693,NaN,1.0,100.0,1.0,0.0
3,PF00001_PF00322,PF00001,PF00322,7tm_1,Endothelin,4,4,31.19,7.365980,78,5GLH,1.000000,0,10.129397,-0.495000,3.0,57.0,3.0,1.0
4,PF00001_PF00339,PF00001,PF00339,7tm_1,Arrestin_N,6,3,18.24,5.657100,38,6TKO,0.500000,3,12.478734,1.637844,5.0,18.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,PF18782_PF00559,PF18782,PF00559,NAD2,Vif,1,1,7.74,3.639300,23,6NIL,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5720,PF18784_PF00071,PF18784,PF00071,CRM1_repeat_2,Ras,34,34,8.76,4.306780,25,3NBZ,1.000000,0,-0.309397,-0.216169,10.0,50.0,9.0,1.0
5721,PF18801_PF00072,PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.961570,22,3Q15,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5722,PF18806_PF00071,PF18806,PF00071,Importin_rep_3,Ras,1,1,0.05,0.237612,7,2X19,1.000000,0,2.846743,-0.351625,2.0,22.0,2.0,1.0


In [40]:
#454 occurrences of synthetic construct domains in DDI_types in the 3did subset
len(df_3did_HuRI_BioPlex_ProtCID) - len(final_subset)

454

In [41]:
final_subset.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_Interactome_ProtCID_metrics_inter_hetero_subset_without_synthetic_constructs.csv', index=False)

In [43]:
df_subset = df_3did_HuRI_BioPlex_ProtCID.filter(['DDI_type', 'Zscore'])

In [45]:
df_subset['Potential synthetic construct'] = np.where(df_subset['DDI_type'].isin(final_subset['DDI_type'].tolist()), 0,1)

In [48]:
df_mc = pd.read_csv('/Users/johgeist/Downloads/DDI_curation table_wihtout_homo_protein_structures_4.csv')
df_mc = df_mc[~df_mc['DDI_type'].str.contains('curation')]
df_mc['PDB_ID'] = df_mc['PDB_ID'].str.upper()

In [50]:
df_subset['Manually curated'] = np.where(df_subset['DDI_type'].isin(df_mc['DDI_type'].tolist()), 1,0)

In [69]:
df_subset.rename({'DDI_type': 'DDI type', 'Zscore': '3did z-score'}, axis=1,inplace=True)

In [70]:
df_subset

,DDI type,3did z-score,Potential synthetic construct,Manually curated
0,PF00001_PF00048,5.101720,0,0
1,PF00001_PF00085,1.963520,0,0
2,PF00001_PF00087,5.828060,0,0
3,PF00001_PF00322,7.365980,0,0
4,PF00001_PF00339,5.657100,0,0
...,...,...,...,...
5719,PF18782_PF00559,3.639300,0,0
5720,PF18784_PF00071,4.306780,0,0
5721,PF18801_PF00072,4.961570,0,0
5722,PF18806_PF00071,0.237612,0,0


In [71]:
df_subset.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_subset_hetero-protein_interachain.csv')